In [ ]:
''' IMPORTAR INSTALAR LIBRERIAS '''

In [1]:
pip install catboost

In [2]:
pip install shap

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   - -------------------------------------- 3.9/150.0 MB 39.0 MB/s eta 0:00:04
   --- ------------------------------------ 11.3/150.0 MB 35.2 MB/s eta 0:00:04
   ----- ---------------------------------- 18.9/150.0 MB 35.1 MB/s eta 0:00:04
   ------- -------------------------------- 28.0/150.0 MB 38.6 MB/s eta 0:00:04
   --------- ------------------------------ 35.7/150.0 MB 38.4 MB/s eta 0:00:03
   ----------- ---------------------------- 43.8/150.0 MB 38.1 MB/s eta 0:00:03
   ------------- -------------------------- 51.9/150.0 MB 38.0 MB/s eta 0:00:03
   ---------------- ----------------------- 60.0/150.0 MB 37.9 MB/s eta 0:00:03
   ------------------ --------------------- 68.2/150.0 MB 37.8 MB/s eta 0:00:03
   -------------------- ------------------- 76.0/150.0 MB 38.2 MB/s eta 0:00:02
   --------------------- ------------------ 82.3/150.0 MB 3

In [5]:
# gestión de datos
import numpy as np
import pandas as pd

# fechas
import datetime as dt
from datetime import datetime
from dateutil.relativedelta import relativedelta

# gráficos
import seaborn as sns
import matplotlib.pyplot as plt

# preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# modelos
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# metricas
import sklearn.metrics as metrics

# interpretabilidad
import shap

# hiperparametrizado
from sklearn.model_selection import GridSearchCV

# quitar alertas
import warnings
warnings.filterwarnings("ignore")

c:\Users\rafas\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
''' DATOS '''

' DATOS '

In [15]:
df = pd.read_csv('./DATA/viviendas_regresion_basica.csv')

Nuestro cliente nos pide que le digamos el precio al que podemos esperar vender las 3 últimas viviendas que acaban de entrar en la base de datos, y con qué margen de seguridad nos movemos. Para ello, nos dan los datos de las viviendas que tienen en su base de datos.

In [16]:
df

,id_vivienda,superficie,num_hab,distancia,fecha_const,parking,premiada,tam_buzon,precio,impuestos,vendida
0,viv_1,65.0,1,7,1991-01-01,ext,0,0.9,166000.0,16600.0,1.0
1,viv_2,97.0,4+,8,1994-01-01,ext,0,1.8,279000.0,27900.0,1.0
2,viv_3,80.0,4+,4,1970-01-01,NaN,0,1.0,233000.0,23300.0,1.0
3,viv_4,73.0,1,1,1983-01-01,ext,0,1.1,182000.0,18200.0,1.0
4,viv_5,86.0,2,8,1999-01-01,NaN,0,1.2,210000.0,21000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
6998,viv_6999,128.0,3,5,1979-01-01,NaN,0,1.3,317000.0,31700.0,0.0
6999,viv_7000,77.0,4+,7,1996-01-01,ext,0,1.5,177000.0,17700.0,0.0
7000,viv_7001,128.0,3,6,2022-01-01,ext,0,0.7,NaN,NaN,NaN
7001,viv_7002,93.0,4+,9,1988-01-01,NaN,0,1.5,NaN,NaN,NaN


In [17]:
df_pred = df.tail(3)

In [18]:
df_pred

,id_vivienda,superficie,num_hab,distancia,fecha_const,parking,premiada,tam_buzon,precio,impuestos,vendida
7000,viv_7001,128.0,3,6,2022-01-01,ext,0,0.7,NaN,NaN,NaN
7001,viv_7002,93.0,4+,9,1988-01-01,NaN,0,1.5,NaN,NaN,NaN
7002,viv_7003,68.0,2,3,1972-01-01,NaN,0,0.7,NaN,NaN,NaN


In [19]:
df = df.drop(df.tail(3).index)

In [22]:
df["vendida"].value_counts()

vendida
1.0    5000
0.0    2000
Name: count, dtype: int64

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_vivienda  7000 non-null   object 
 1   superficie   6990 non-null   float64
 2   num_hab      7000 non-null   object 
 3   distancia    7000 non-null   int64  
 4   fecha_const  7000 non-null   object 
 5   parking      4763 non-null   object 
 6   premiada     7000 non-null   int64  
 7   tam_buzon    7000 non-null   float64
 8   precio       7000 non-null   float64
 9   impuestos    7000 non-null   float64
 10  vendida      7000 non-null   float64
dtypes: float64(5), int64(2), object(4)
memory usage: 601.7+ KB
